## Shared Imports

In [2]:
from dataHandler.importer import updateDB
from config import NETWORK, USERNAME, DBFILE

import ipywidgets as widgets

import matplotlib.pyplot as plt

## Update Database

In [2]:
b = widgets.Button(description='Update Database',tooltip='call default updateDB()', icon='cloud-download')
display(b)
b.on_click(updateDB(DBFILE, NETWORK, USERNAME))

Button(description='Update Database', icon='cloud-download', style=ButtonStyle(), tooltip='call default update…

KeyboardInterrupt: 

## Temporary Section

In [3]:
import pandas as pd
from dataHandler.models import Base, Scrobble
import datetime as dt

__create db session__

In [4]:
# this section should be made into a decorator
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
engine = create_engine(('sqlite:///' + DBFILE), echo=False)
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
session = Session()

In [5]:
def plot_artist(artist):
    query = session.query(Scrobble).filter(Scrobble.artist == artist)
    df = pd.read_sql(query.statement,session.bind)
    df['timestamp'] = df['timestamp'].apply(lambda x: dt.datetime.utcfromtimestamp(int(x)))
    f_table = df.groupby(pd.Grouper(key='timestamp', freq='M'))['track'].count()
    
    plt.plot(f_table)
    plt.ylabel('Scrobbles / Month')
    plt.title('Scrobbles of `' + artist + '` per Month')
    x_ticks = f_table.index.values
    x_labels = f_table.index.month_name().values
    plt.xticks(x_ticks, x_labels, rotation=45)
    display(plt.show())

In [6]:
def get_song_scrobbles(song, artist):
    query = session.query(Scrobble).filter(Scrobble.track == song).filter(Scrobble.artist == artist)
    df = pd.read_sql(query.statement, session.bind)
    df['timestamp'] = df['timestamp'].apply(lambda x: dt.datetime.utcfromtimestamp(int(x)))
    df = df.sort_values(by='timestamp').reset_index(drop=True)
    plt.plot(df['timestamp'],df.index.values, label=song+', '+artist)

In [7]:
# get_song_scrobbles('Dancing On My Own', 'Calum Scott')
# get_song_scrobbles('Scars', 'Alfie Arcuri')
# get_song_scrobbles('Bloodstain', 'Wrabel')
# get_song_scrobbles('Break My Heart Again', 'FINNEAS')
# get_song_scrobbles('In My Head', 'Peter Manos')
# get_song_scrobbles('Rewrite The Stars', 'Zac Efron')
# get_song_scrobbles('Haze', 'Amber Run')
# get_song_scrobbles('I Gave It All', 'Aquilo')
# get_song_scrobbles('You Are The Reason', 'Calum Scott')
# get_song_scrobbles('Say', 'Will Heggadon')
# ----
get_song_scrobbles('Dancing On My Own', 'Calum Scott')
get_song_scrobbles('Bloodstain', 'Wrabel')
get_song_scrobbles('Look Away', 'Eli Lieb')
get_song_scrobbles('Walking','Steve Grand')
get_song_scrobbles('timekeeper','Greyson Chance')
get_song_scrobbles('Shut Up','Greyson Chance')
get_song_scrobbles('black on black','Greyson Chance')
# get_song_scrobbles('Hold Each Other', 'A Great Big World')
#---
plt.legend(bbox_to_anchor=(1.04,1), loc="upper left")
plt.xticks(rotation=45)
plt.show()

In [8]:
artist = ''
def set_artist(input):
    global artist
    artist = input.value

In [9]:
t = widgets.Text(description='Artist:')
display(t)
t.on_submit(set_artist)

In [10]:
plot_artist(artist)

TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Index'

In [11]:
session.close()